# ICLR Workshop Challenge #1: CGIAR Computer Vision for Crop Disease

Amritha Musipatla and Grisam Shah  
CU_sm3773_gvs2110

## Imports and packages

## Preprocessing

In this section we format the given train and test image sets to standard form. 

In [2]:
# Read images into array


# Resize to standard shape